In [ ]:
!pip install mne
!pip install numpy scipy matplotlib scikit-learn
!pip install pactools
!pip install mne_icalabel

In [ ]:
import mne
import numpy as np
import os.path as op
import matplotlib.pyplot as plt

from pactools import simulate_pac, raw_to_mask, Comodulogram, MaskIterator

import mne_icalabel

In [ ]:
raw = mne.io.read_raw_gdf('/content/drive/MyDrive/EEG Raw Data/participantID.gdf')
events = mne.find_events(raw, stim_channel="STATUS")
events[:, 2] >>= 8
raw2 = raw.set_montage('biosemi128')

print(raw2.info)

if events.size == 0:
    print("The 'events' array is empty.")

tmin = events[0, 0]/raw2.info['sfreq']
tmax = events[1, 0]/raw2.info['sfreq']

raw2.crop(tmin=tmin, tmax=tmax)
raw2.load_data()

In [ ]:
eeg = raw2.filter(l_freq=1, h_freq=70)

filtered = eeg.plot(duration=300, proj=False, n_channels=len(raw.ch_names))

start_events = events[events[:, 2] == 18]
end_events = events[events[:, 2] == 28]

In [ ]:
# check if start_events array is empty or not
if len(start_events) > 0:
    print("start_events contains elements.")
else:
    print("start_events is empty.")

# Create new_events array with only one epoch
if len(start_events) > 0:
    epoch_start = start_events[0, 0]  # Start time of the first event
    epoch_end = epoch_start + 300  # Duration of one epoch (3 seconds)
    new_events = np.array([[epoch_start, 0, 18]])  # Add only one epoch to the array
else:
    new_events = np.array([])  # Empty array if start_events is empty

print(new_events)

In [ ]:
epochs = mne.Epochs(eeg, new_events, event_id=None, tmin=0, tmax=300, baseline=None)
epochs.load_data()
epochs.resample(256)

epochs.set_eeg_reference(ref_channels='average')

In [ ]:
mu = 1
sigma = 0.25
tmin, tmax = mu - 3 * sigma, mu + 3 * sigma

#set index for electrodes in left hemisphere
c9_idx = raw.ch_names.index('C9')
a10_idx = raw.ch_names.index('A10')
ixs = (c9_idx, a10_idx)


# create the input array for Comodulogram.fit
low_sig, high_sig, mask = raw_to_mask(eeg, ixs=ixs, events=new_events, tmin=tmin,
                                      tmax=tmax)

In [ ]:
low_sig = eeg[ixs[0], :][0]
high_sig = eeg[ixs[1], :][0]

mask = MaskIterator(new_events, tmin, tmax, eeg.n_times, eeg.info['sfreq'])

# create the instance of Comodulogram
estimator = Comodulogram(fs=eeg.info['sfreq'],
                         low_fq_range=np.linspace(1, 10, 100), low_fq_width=2.,
                         method='duprelatour', progress_bar=True)
# compute the comodulogram
estimator.fit(low_sig, high_sig, mask)
# plot the results
fig = estimator.plot(tight_layout=False)
plt.ylim(0, 100)
plt.show()

In [ ]:
print("Comodulogram shape:", estimator.comod_.shape)

In [ ]:
# Extract the peak PAC metric from the comodulogram
peak_index = np.argmax(estimator.comod_)
peak_freq_index, peak_time_index = np.unravel_index(peak_index, estimator.comod_.shape[1:])
peak_freq = estimator.low_fq_range[peak_freq_index]

print("Peak PAC frequency:", peak_freq)

In [ ]:
# Extract the PAC metric associated with the peak PAC frequency
peak_pac_metric = estimator.comod_[0, peak_freq_index, peak_time_index]

print("PAC metric at peak PAC frequency:", peak_pac_metric)

In [ ]:
#set index for electrodes in right hemisphere
c9_idx = raw.ch_names.index('C9')
c11_idx = raw.ch_names.index('C11')
ixs = (c9_idx, c11_idx)

low_sig = eeg[ixs[0], :][0]
high_sig = eeg[ixs[1], :][0]

mask = MaskIterator(new_events, tmin, tmax, eeg.n_times, eeg.info['sfreq'])

# create the instance of Comodulogram
estimator = Comodulogram(fs=eeg.info['sfreq'],
                         low_fq_range=np.linspace(1, 10, 100), low_fq_width=2,
                         method='duprelatour', progress_bar=True)
# compute the comodulogram
estimator.fit(low_sig, high_sig, mask)
# plot the results
fig = estimator.plot(tight_layout=False)
plt.ylim(0, 100)
plt.show()

In [ ]:
# Extract the peak PAC metric from the comodulogram
peak_index = np.argmax(estimator.comod_)
peak_freq_index, peak_time_index = np.unravel_index(peak_index, estimator.comod_.shape[1:])
peak_freq = estimator.low_fq_range[peak_freq_index]

print("Peak PAC frequency:", peak_freq)

In [ ]:
# Extract the PAC metric associated with the peak PAC frequency
peak_pac_metric = estimator.comod_[0, peak_freq_index, peak_time_index]

print("PAC metric at peak PAC frequency:", peak_pac_metric)